# Import libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import matplotlib.ticker as ticker
import plotly.graph_objects as go
from matplotlib.ticker import MaxNLocator

import time
from sklearn.preprocessing import LabelEncoder

import numpy as np
import optuna

import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Set notebook environment

In [2]:
# Change notebook environment

# https://ipython.readthedocs.io/en/stable/api/generated/IPython.core.interactiveshell.html
# https://stackoverflow.com/questions/36786722/how-to-display-full-output-in-jupyter-not-only-last-result
# Display full output in output cell, not only last result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# https://www.kaggle.com/questions-and-answers/118932
# Max rows and columns of pandas dataframe 
pd.options.display.max_rows,pd.options.display.max_columns

# https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
# Print all the contents of a pandas dataframe
pd.set_option('display.max_rows', None) # Print unlimited number of rows by setting to None, default is 10
pd.set_option('display.max_columns', None) # Do not truncate columns to display all of them by setting to None
pd.set_option('display.width', None) # Auto-detect the width of dataframe to display all columns in single line by setting to None
pd.set_option('display.max_colwidth', None) # Auto detect the max size of column and print contents of that column without truncation

# https://stackoverflow.com/questions/8924173/how-to-print-bold-text-in-python
start = "\033[1m" # Bold text
end = "\033[0;0m" # Reset 

import gc

In [3]:
# Free up memory that is no longer being used
gc.collect()

60

# Load datasets

In [4]:
train_df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
greeks_df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")
test_df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
sample_submission_df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

# Data glance

In [5]:
print(start+"train.csv:"+end)
train_df.head()
print(start+"\ngreeks.csv:"+end)
greeks_df.head()
print(start+"\ntest.csv:"+end)
test_df.head()
print(start+"\nsample_submission_df.csv:"+end)
sample_submission_df.head()

train.csv:


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1



greeks.csv:


,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020



test.csv:


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



sample_submission_df.csv:


,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5


# Check column names

In [6]:
print(start+"train.csv:"+end)
train_df.columns
print(start+"\ngreeks.csv:"+end)
greeks_df.columns
print(start+"\ntest.csv:"+end)
test_df.columns
print(start+"\nsample_submission_df.csv:"+end)
sample_submission_df.columns

train.csv:


Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'Class'],
      dtype='object')


greeks.csv:


Index(['Id', 'Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon'], dtype='object')


test.csv:


Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
      dtype='object')


sample_submission_df.csv:


Index(['Id', 'class_0', 'class_1'], dtype='object')

# Assess data
- Attribute
- Summary statistics

In [7]:
print(start+"train:", train_df.shape,end)
print(start+"greeks:", greeks_df.shape,end)
print(start+"test:", test_df.shape,end)
print(start+"sample submission:", sample_submission_df.shape,end)

train: (617, 58) 
greeks: (617, 6) 
test: (5, 57) 
sample submission: (5, 3) 


In [8]:
print(start+"train.csv:"+end)
train_df.info()
print(start+"\ngreeks_df:"+end)
greeks_df.info()
print(start+"\ntest_df:"+end)
test_df.info()
print(start+"\nsample_submission_df:"+end)
sample_submission_df.info()

train.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 58 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    floa

In [9]:
print(start+"train_df:"+end)
train_df.describe()
print(start+"\ngreeks_df:"+end)
greeks_df.describe()
print(start+"\ntest_df:"+end)
test_df.describe()
print(start+"\nsample_submission_df:"+end)
sample_submission_df.describe()

train_df:


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,557.000000,617.000000,617.000000,615.000000,614.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,557.000000,617.000000,617.000000,616.000000,617.000000,617.000000,617.000000,616.000000,617.000000,615.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000
mean,0.477149,3502.013221,118.624513,38.968552,10.128242,5.545576,0.060320,10.566447,8.053012,5350.388655,21.419492,231.322223,98.328737,1218.133238,550.632525,77.104151,0.688801,90.251735,11.241064,0.030615,1.403761,0.742262,36.917590,1.383792,27.165653,51.128326,401.901299,0.633884,0.367002,146.972099,94.795377,26.370568,1.802900,1.924830,26.388989,9.072700,3.064778,1731.248215,0.305107,69.582596,105.060712,69.117005,71.341526,6.930086,10306.810737,10.111079,5.433199,3.533905,0.421501,20.724856,131.714987,14679.595398,31.489716,50.584437,8.530961,0.175041
std,0.468388,2300.322717,127.838950,69.728226,10.518877,2.551696,0.416817,4.350645,65.166943,3021.326641,3.478278,183.992505,96.479371,7575.293707,2076.371275,159.049302,0.263994,51.585130,13.571133,0.014808,1.922210,0.281195,17.266347,0.538717,14.645993,21.210888,317.745623,1.912384,0.112989,86.084419,28.243187,8.038825,9.034721,1.484555,18.116679,6.200281,2.058344,1790.227476,1.847499,38.555707,68.445620,390.187057,165.551545,64.754262,11331.294051,2.934025,11.496257,50.181948,1.305365,9.991907,144.181524,19352.959387,9.864239,36.266251,10.327010,0.380310
min,0.081187,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,9.886800,72.948951,1.331155,51.216883,257.432377,12.499760,0.176874,23.387600,0.510888,0.003184,1.050225,0.069225,13.784111,0.137925,7.030640,6.906400,35.998895,0.238680,0.040995,60.232470,10.345600,6.339496,0.005518,1.743070,0.804068,4.926396,0.286201,185.594100,0.003042,5.394675,78.526968,3.828384,7.534128,0.296850,1563.136688,3.583450,0.173229,0.497060,0.067730,4.102182,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.252107,2197.345480,85.200147,12.270314,8.138688,4.128294,0.025578,8.129580,1.229900,4155.702870,19.420500,156.847239,27.834425,424.990642,257.432377,23.317567,0.563688,64.724192,5.066306,0.023482,1.050225,0.589575,29.782467,1.070298,7.030640,37.942520,188.815690,0.238680,0.295164,102.703553,78.232240,20.888264,0.005518,1.743070,14.715792,5.965392,1.648679,1111.160625,0.003042,30.927468,78.526968,4.324656,25.815384,0.296850,5164.666260,8.523098,0.173229,0.497060,0.067730,14.036718,72.611063,2798.992584,25.034888,23.011684,0.124392,0.000000
50%,0.354659,3120.318960,85.200147,20.533110,8.138688,5.031912,0.025578,10.461320,1.229900,4997.960730,21.186000,193.908816,61.642115,627.417402,257.432377,42.554330,0.658715,79.819104,9.123000,0.027860,1.050225,0.730800,34.835130,1.351665,36.019104,49.180940,307.509595,0.238680,0.358023,130.050630,96.264960,25.248800,0.251741,1.743070,21.642456,8.149404,2.616119,1493.817413,0.085176,71.949306,78.526968,22.641144,36.394008,1.870155,7345.143424,9.945452,3.028141,1.131000,0.250601,18.771436,72.611063,7838.273610,30.608946,41.007968,0.337827,0.000000
75%,0.559763,4361.637390,113.739540,39.139886,8.138688,6.431634,0.036845,12.969516,5.081244,6035.885700,23.657700,247.803462,134.009015,975.649259,257.432377,77.310097,0.772206,99.813520,13.565901,0.034427,1.228445,0.859350,40.529401,1.660617,37.935832,61.408760,507.896200,0.238680,0.426348,165.836955,110.640680,30.544224,1.058690,1.743070,34.058344,10.503048,3.910070,1905.701475,0.237276,109.125159,112.766654,49.085352,56.714448,4.88021


greeks_df:


,Id,Alpha,Beta,Gamma,Delta,Epsilon
count,617,617,617,617,617,617
unique,617,4,3,8,4,198
top,000ff2bfdfe9,A,C,M,B,Unknown
freq,1,509,407,445,456,144



test_df:


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
count,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



sample_submission_df:


,class_0,class_1
count,5.0,5.0
mean,0.5,0.5
std,0.0,0.0
min,0.5,0.5
25%,0.5,0.5
50%,0.5,0.5
75%,0.5,0.5
max,0.5,0.5


# Check missing data

In [10]:
# Check for missing values

print(start+'Check missing values:'+end)

print(start+"\ngreeks_df:"+end)
print(greeks_df.isnull().sum())
print(start+"\ntrain_df:"+end)
print(train_df.isnull().sum())
print(start+"\ntest_df:"+end)
print(test_df.isnull().sum())
print(start+"\nsample_submission_df:"+end)
print(sample_submission_df.isnull().sum())

Check missing values:

greeks_df:
Id         0
Alpha      0
Beta       0
Gamma      0
Delta      0
Epsilon    0
dtype: int64

train_df:
Id        0
AB        0
AF        0
AH        0
AM        0
AR        0
AX        0
AY        0
AZ        0
BC        0
BD        0
BN        0
BP        0
BQ       60
BR        0
BZ        0
CB        2
CC        3
CD        0
CF        0
CH        0
CL        0
CR        0
CS        0
CU        0
CW        0
DA        0
DE        0
DF        0
DH        0
DI        0
DL        0
DN        0
DU        1
DV        0
DY        0
EB        0
EE        0
EG        0
EH        0
EJ        0
EL       60
EP        0
EU        0
FC        1
FD        0
FE        0
FI        0
FL        1
FR        0
FS        2
GB        0
GE        0
GF        0
GH        0
GI        0
GL        1
Class     0
dtype: int64

test_df:
Id     0
AB     0
AF     0
AH     0
AM     0
AR     0
AX     0
AY     0
AZ     0
BC     0
BD     0
BN     0
BP     0
BQ     0
BR     0
BZ     0
C

In [11]:
# Keep all rows with at least one missing data
#train_df[train_df.isna().any(axis=1)]

# Check duplication

In [12]:
# Check for duplicates

print(start+'Train set: There are a total of', start+str(train_df.duplicated().sum())+end, start+'duplicate rows.\n'+end)
print(start+'Greeks: There are a total of', start+str(greeks_df.duplicated().sum())+end, start+'duplicate rows.\n'+end)
print(start+'Test set: There are a total of', start+str(test_df.duplicated().sum())+end, start+'duplicate rows.\n'+end)
print(start+'Submission: There are a total of', start+str(sample_submission_df.duplicated().sum())+end, start+'duplicate rows.\n'+end)

Train set: There are a total of 0 duplicate rows.

Greeks: There are a total of 0 duplicate rows.

Test set: There are a total of 0 duplicate rows.

Submission: There are a total of 0 duplicate rows.



In [13]:
# Check duplication by Id
train_df[train_df.Id.duplicated(keep=False)].sort_values("Id")
greeks_df[greeks_df.Id.duplicated(keep=False)].sort_values("Id")

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class


,Id,Alpha,Beta,Gamma,Delta,Epsilon


# Class imbalance

In [14]:
# Create a plot as needed
### YOUR CODE HERE ###

piefreq=train_df.Class.value_counts()

print('Class imbalance (counts): 1=Diagnosed, 0=Not Diagnosed')
print(piefreq,'\n')

# Imbalance class
print('Normalized class: 1=Diagnosed, 0=Not Diagnosed')
print(train_df.Class.value_counts(normalize=True))

# Pie chart of class imbalance
fig=go.Figure(data=[go.Pie(labels=['Not Diagnosed    (n=' + str(piefreq[0]) +')',
                                   'Diagnosed           (n=' + str(piefreq[1]) +')'],
                           values=train_df.Class.value_counts())])
fig.update_layout(title=dict(text="<b>Pie chart of 'Class'</b>",
                             y=0.85,x=0.4,
                             xanchor='center',
                             yanchor='top',
                             font=dict(size=14)
                            )
                  ,width=600
                  ,height=600)

# Delete piefreq dataframe to release memory
del piefreq, fig

Class imbalance (counts): 1=Diagnosed, 0=Not Diagnosed
0    509
1    108
Name: Class, dtype: int64 

Normalized class: 1=Diagnosed, 0=Not Diagnosed
0    0.824959
1    0.175041
Name: Class, dtype: float64


# Merge datasets by Id
- Train and Greek 

In [15]:
# Merge train and greeks dataframe by Id
merged_df = pd.merge(train_df, greeks_df, on="Id")

In [16]:
merged_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1,B,C,G,D,3/19/2019
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0,A,C,M,B,Unknown
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0,A,C,M,B,Unknown
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0,A,C,M,B,Unknown
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1,D,B,F,B,3/25/2020


In [17]:
# Categorical feature
merged_df.EJ.value_counts()

B    395
A    222
Name: EJ, dtype: int64

In [18]:
# Categorical feature
print(start+"Alpha:"+end)
greeks_df[~greeks_df["Alpha"].str.isnumeric()]["Alpha"].value_counts()
print("\n"+start+"Beta:"+end)
greeks_df[~greeks_df["Beta"].str.isnumeric()]["Beta"].value_counts()
print("\n"+start+"Gamma:"+end)
greeks_df[~greeks_df["Gamma"].str.isnumeric()]["Gamma"].value_counts()
print("\n"+start+"Delta:"+end)
greeks_df[~greeks_df["Delta"].str.isnumeric()]["Delta"].value_counts()
print("\n"+start+"Epsilon:"+end)
greeks_df[~greeks_df["Epsilon"].str.isnumeric()]["Epsilon"].value_counts()

Alpha:


A    509
B     61
G     29
D     18
Name: Alpha, dtype: int64


Beta:


C    407
B    202
A      8
Name: Beta, dtype: int64


Gamma:


M    445
N     64
H     53
B     18
A     11
F     10
G      8
E      8
Name: Gamma, dtype: int64


Delta:


B    456
A     75
C     64
D     22
Name: Delta, dtype: int64


Epsilon:


Unknown       144
5/29/2019      24
6/26/2019      19
9/15/2020      17
8/29/2019      13
4/24/2019      11
9/8/2020       11
1/29/2019      10
1/31/2019      10
2/18/2019      10
4/30/2020       9
9/13/2020       8
7/30/2020       7
7/27/2020       6
6/11/2020       5
9/14/2020       5
11/27/2018      5
11/25/2019      5
9/11/2020       5
3/1/2019        5
12/10/2018      5
7/28/2020       5
5/6/2020        5
3/15/2019       5
2/28/2019       5
5/27/2020       4
6/21/2019       4
10/1/2019       4
4/29/2020       4
11/30/2018      4
12/4/2018       4
12/20/2018      4
3/13/2019       4
7/24/2020       4
4/28/2020       4
6/2/2020        4
9/7/2020        4
9/16/2020       3
7/16/2020       3
7/29/2020       3
12/24/2019      3
7/19/2019       3
3/7/2019        3
12/30/2019      3
4/9/2019        3
12/19/2019      3
12/9/2019       3
3/28/2019       2
5/2/2020        2
7/30/2019       2
7/15/2020       2
7/15/2019       2
1/28/2020       2
6/25/2020       2
2/22/2019       2
10/18/2019

In [19]:
# Print counts of 'Unknown' Epsilon
unknown_df=greeks_df.query("Epsilon=='Unknown'")
unknown_df.value_counts()

Id            Alpha  Beta  Gamma  Delta  Epsilon
007255e47698  A      C     M      B      Unknown    1
013f2bd269f5  A      C     M      B      Unknown    1
a7b1c7b83f53  A      C     M      B      Unknown    1
ab39fd701211  A      C     M      B      Unknown    1
ab4bc55cb79e  A      C     M      B      Unknown    1
adb67a0dd661  A      C     M      B      Unknown    1
aee064d35f72  A      C     M      B      Unknown    1
af0802c15f01  A      C     M      B      Unknown    1
afad421356e3  A      C     M      B      Unknown    1
b06b043e7abe  A      C     M      B      Unknown    1
b1056bf99b86  A      C     M      B      Unknown    1
b10bae274dfc  A      C     M      B      Unknown    1
b10f6f101138  A      C     M      B      Unknown    1
b301f38c110c  A      C     M      B      Unknown    1
b4d611fab6c5  A      C     M      B      Unknown    1
b874e25c20aa  A      C     M      B      Unknown    1
ba0c42e12f1f  A      C     M      B      Unknown    1
a633a258eeed  A      C     M     

# Feature Encoding

In [20]:
le = LabelEncoder()

# Columns to encode
cat_features = ["EJ", "Alpha", "Beta", "Gamma", "Delta"] 
enc_df=merged_df.copy()

# Apply label encoder to each column
enc_df[cat_features] = enc_df[cat_features].apply(le.fit_transform) 

enc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 617 entries, 0 to 616
Data columns (total 63 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Id       617 non-null    object 
 1   AB       617 non-null    float64
 2   AF       617 non-null    float64
 3   AH       617 non-null    float64
 4   AM       617 non-null    float64
 5   AR       617 non-null    float64
 6   AX       617 non-null    float64
 7   AY       617 non-null    float64
 8   AZ       617 non-null    float64
 9   BC       617 non-null    float64
 10  BD       617 non-null    float64
 11  BN       617 non-null    float64
 12  BP       617 non-null    float64
 13  BQ       557 non-null    float64
 14  BR       617 non-null    float64
 15  BZ       617 non-null    float64
 16  CB       615 non-null    float64
 17  CC       614 non-null    float64
 18  CD       617 non-null    float64
 19  CF       617 non-null    float64
 20  CH       617 non-null    float64
 21  CL       617 non

In [21]:
'''
print(start+"Before"+end)
train_df['EJ'].value_counts()
enc_df['EJ'].value_counts()

greeks_df['Alpha'].value_counts()
enc_df['Alpha'].value_counts()

greeks_df['Beta'].value_counts()
enc_df['Beta'].value_counts()

greeks_df['Gamma'].value_counts()
enc_df['Gamma'].value_counts()

greeks_df['Delta'].value_counts()
enc_df['Delta'].value_counts()
'''

'\nprint(start+"Before"+end)\ntrain_df[\'EJ\'].value_counts()\nenc_df[\'EJ\'].value_counts()\n\ngreeks_df[\'Alpha\'].value_counts()\nenc_df[\'Alpha\'].value_counts()\n\ngreeks_df[\'Beta\'].value_counts()\nenc_df[\'Beta\'].value_counts()\n\ngreeks_df[\'Gamma\'].value_counts()\nenc_df[\'Gamma\'].value_counts()\n\ngreeks_df[\'Delta\'].value_counts()\nenc_df[\'Delta\'].value_counts()\n'

# Modeling

In [22]:
def balanced_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)

In [23]:
def balanced_log_loss_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    bll = balanced_log_loss(y_true, y_pred)
    return 'balanced_log_loss', bll, False

In [24]:
# Drop features not found in test data, Separate the features and target variable
X_df = enc_df.drop(["Id", "Alpha", "Beta", "Gamma", "Delta","Epsilon","Class"], axis=1)
y_df = enc_df['Class']

In [25]:
def objective(trial):
    train_x,valid_x,train_y,valid_y=train_test_split(X_df,y_df,test_size=0.2,random_state=42,shuffle=True,stratify=y_df)
    
    dtrain = lgb.Dataset(train_x,label=train_y,
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])
    dvalid = lgb.Dataset(valid_x,label=valid_y,
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])
    
    param = {"objective": "binary",
             "metric": "binary_logloss",
             "verbosity": -1,
             "boosting_type": "gbdt",
             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
             "seed":42,
             "use_missing":True}
  
    # Train the LightGBM model with the custom evaluation function
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid],feval=balanced_log_loss_eval,categorical_feature=["EJ"])
    preds = gbm.predict(valid_x)
    
    bll = balanced_log_loss(valid_y,preds)
    
    return bll


if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-07-24 20:36:20,994] A new study created in memory with name: no-name-5b578597-7ed2-4105-8621-8223f3e66071


[1]	valid_0's binary_logloss: 0.452982	valid_0's balanced_log_loss: 0.54387
[2]	valid_0's binary_logloss: 0.438616	valid_0's balanced_log_loss: 0.521944
[3]	valid_0's binary_logloss: 0.43017	valid_0's balanced_log_loss: 0.50763
[4]	valid_0's binary_logloss: 0.427036	valid_0's balanced_log_loss: 0.504821
[5]	valid_0's binary_logloss: 0.419281	valid_0's balanced_log_loss: 0.493824
[6]	valid_0's binary_logloss: 0.415616	valid_0's balanced_log_loss: 0.487167
[7]	valid_0's binary_logloss: 0.411137	valid_0's balanced_log_loss: 0.482108
[8]	valid_0's binary_logloss: 0.403332	valid_0's balanced_log_loss: 0.471334
[9]	valid_0's binary_logloss: 0.399541	valid_0's balanced_log_loss: 0.4637
[10]	valid_0's binary_logloss: 0.389333	valid_0's balanced_log_loss: 0.45164
[11]	valid_0's binary_logloss: 0.380571	valid_0's balanced_log_loss: 0.439111
[12]	valid_0's binary_logloss: 0.377128	valid_0's balanced_log_loss: 0.438778
[13]	valid_0's binary_logloss: 0.370765	valid_0's balanced_log_loss: 0.425602
[

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.

[I 2023-07-24 20:36:21,084] Trial 0 finished with value: 0.263976882305519 and parameters: {'lambda_l1': 4.3498524285600205e-08, 'lambda_l2': 2.2779773802078477e-05, 'num_leaves': 17, 'feature_fraction': 0.7306012532158899, 'bagging_fraction': 0.5800345547111445, 'bagging_freq': 1, 'min_child_samples': 88}. Best is trial 0 with value: 0.263976882305519.


[58]	valid_0's binary_logloss: 0.248681	valid_0's balanced_log_loss: 0.281076
[59]	valid_0's binary_logloss: 0.246778	valid_0's balanced_log_loss: 0.276827
[60]	valid_0's binary_logloss: 0.24429	valid_0's balanced_log_loss: 0.274798
[61]	valid_0's binary_logloss: 0.246648	valid_0's balanced_log_loss: 0.279726
[62]	valid_0's binary_logloss: 0.245796	valid_0's balanced_log_loss: 0.279021
[63]	valid_0's binary_logloss: 0.243843	valid_0's balanced_log_loss: 0.278383
[64]	valid_0's binary_logloss: 0.241818	valid_0's balanced_log_loss: 0.274119
[65]	valid_0's binary_logloss: 0.241927	valid_0's balanced_log_loss: 0.274788
[66]	valid_0's binary_logloss: 0.243427	valid_0's balanced_log_loss: 0.276582
[67]	valid_0's binary_logloss: 0.242809	valid_0's balanced_log_loss: 0.275234
[68]	valid_0's binary_logloss: 0.24278	valid_0's balanced_log_loss: 0.274457
[69]	valid_0's binary_logloss: 0.239212	valid_0's balanced_log_loss: 0.271031
[70]	valid_0's binary_logloss: 0.239762	valid_0's balanced_log_los

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.

[I 2023-07-24 20:36:21,175] Trial 1 finished with value: 0.3088843320871283 and parameters: {'lambda_l1': 0.00014720394834549238, 'lambda_l2': 0.577698857938498, 'num_leaves': 206, 'feature_fraction': 0.6903459088894963, 'bagging_fraction': 0.5704699080626566, 'bagging_freq': 5, 'min_child_samples': 82}. Best is trial 0 with value: 0.263976882305519.
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.



...

[1]	valid_0's binary_logloss: 0.439642	valid_0's balanced_log_loss: 0.525467
[2]	valid_0's binary_logloss: 0.422573	valid_0's balanced_log_loss: 0.50206
[3]	valid_0's binary_logloss: 0.404396	valid_0's balanced_log_loss: 0.476855
[4]	valid_0's binary_logloss: 0.39001	valid_0's balanced_log_loss: 0.456644
[5]	valid_0's binary_logloss: 0.378214	valid_0's balanced_log_loss: 0.440713
[6]	valid_0's binary_logloss: 0.367564	valid_0's balanced_log_loss: 0.427651
[7]	valid_0's binary_logloss: 0.356908	valid_0's balanced_log_loss: 0.413749
[8]	valid_0's binary_logloss: 0.348711	valid_0's balanced_log_loss: 0.403812
[9]	valid_0's binary_logloss: 0.34292	valid_0's balanced_log_loss: 0.395238
[10]	valid_0's binary_logloss: 0.336558	valid_0's balanced_log_loss: 0.38871
[11]	valid_0's binary_logloss: 0.331408	valid_0's balanced_log_loss: 0.382988
[12]	valid_0's binary_logloss: 0.32883	valid_0's balanced_log_loss: 0.379229
[13]	valid_0's binary_logloss: 0.324076	valid_0's balanced_log_loss: 0.370968


[I 2023-07-24 20:36:38,041] Trial 98 finished with value: 0.22707688850130156 and parameters: {'lambda_l1': 1.5427527860480176e-06, 'lambda_l2': 0.013121933923043498, 'num_leaves': 169, 'feature_fraction': 0.7093896188931313, 'bagging_fraction': 0.9080488874192538, 'bagging_freq': 7, 'min_child_samples': 57}. Best is trial 58 with value: 0.20253093851793588.
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.



[20]	valid_0's binary_logloss: 0.289134	valid_0's balanced_log_loss: 0.331044
[21]	valid_0's binary_logloss: 0.281866	valid_0's balanced_log_loss: 0.323492
[22]	valid_0's binary_logloss: 0.28037	valid_0's balanced_log_loss: 0.321134
[23]	valid_0's binary_logloss: 0.277927	valid_0's balanced_log_loss: 0.318288
[24]	valid_0's binary_logloss: 0.272364	valid_0's balanced_log_loss: 0.313711
[25]	valid_0's binary_logloss: 0.270298	valid_0's balanced_log_loss: 0.312049
[26]	valid_0's binary_logloss: 0.267716	valid_0's balanced_log_loss: 0.309692
[27]	valid_0's binary_logloss: 0.266673	valid_0's balanced_log_loss: 0.308113
[28]	valid_0's binary_logloss: 0.262568	valid_0's balanced_log_loss: 0.303216
[29]	valid_0's binary_logloss: 0.258908	valid_0's balanced_log_loss: 0.298281
[30]	valid_0's binary_logloss: 0.258622	valid_0's balanced_log_loss: 0.298261
[31]	valid_0's binary_logloss: 0.25761	valid_0's balanced_log_loss: 0.296962
[32]	valid_0's binary_logloss: 0.256267	valid_0's balanced_log_los

[I 2023-07-24 20:36:38,213] Trial 99 finished with value: 0.21664305509535944 and parameters: {'lambda_l1': 2.267159431758931e-07, 'lambda_l2': 0.34816228862812626, 'num_leaves': 152, 'feature_fraction': 0.7615061407566838, 'bagging_fraction': 0.8926118096311829, 'bagging_freq': 6, 'min_child_samples': 71}. Best is trial 58 with value: 0.20253093851793588.


[55]	valid_0's binary_logloss: 0.213227	valid_0's balanced_log_loss: 0.236619
[56]	valid_0's binary_logloss: 0.211649	valid_0's balanced_log_loss: 0.236349
[57]	valid_0's binary_logloss: 0.211578	valid_0's balanced_log_loss: 0.235555
[58]	valid_0's binary_logloss: 0.21015	valid_0's balanced_log_loss: 0.234814
[59]	valid_0's binary_logloss: 0.210617	valid_0's balanced_log_loss: 0.236668
[60]	valid_0's binary_logloss: 0.208061	valid_0's balanced_log_loss: 0.233396
[61]	valid_0's binary_logloss: 0.207512	valid_0's balanced_log_loss: 0.232637
[62]	valid_0's binary_logloss: 0.209517	valid_0's balanced_log_loss: 0.235808
[63]	valid_0's binary_logloss: 0.208905	valid_0's balanced_log_loss: 0.234819
[64]	valid_0's binary_logloss: 0.209931	valid_0's balanced_log_loss: 0.23653
[65]	valid_0's binary_logloss: 0.209732	valid_0's balanced_log_loss: 0.23649
[66]	valid_0's binary_logloss: 0.207482	valid_0's balanced_log_loss: 0.23381
[67]	valid_0's binary_logloss: 0.206943	valid_0's balanced_log_loss:

# LightGBMTunerCV

In [26]:
import optuna.integration.lightgbm as lgb

from lightgbm import early_stopping
from lightgbm import log_evaluation
import sklearn.datasets
#from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    dtrain = lgb.Dataset(X_df, label=y_df,
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])

    param = {"objective": "binary",
             "metric": "binary_logloss",
             "verbosity": -1,
             "boosting_type": "gbdt",
             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
             "seed":42,
             "use_missing":True}
        
    tuner = lgb.LightGBMTunerCV(param,
                                dtrain,
                                folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                                callbacks=[early_stopping(100), 
                                           log_evaluation(100)],
                                optuna_seed=42,
                                feval=balanced_log_loss_eval,
                                feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                              'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                              'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                              'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                              'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                                categorical_feature=["EJ"]
                               )
    '''
    params, train_set, num_boost_round=1000, folds=None, nfold=5, stratified=True, shuffle=True, fobj=None, feval=None, 
    feature_name='auto', categorical_feature='auto', early_stopping_rounds=None, fpreproc=None, verbose_eval=None, show_stdv=True, 
    seed=0, callbacks=None, time_budget=None, sample_size=None, study=None, optuna_callbacks=None, 
    verbosity=None, show_progress_bar=True, model_dir=None, return_cvbooster=False, *, optuna_seed=None)
    '''    
    tuner.run()

    print("Best score:", tuner.best_score)
    best_params = tuner.best_params
    print("Best params:", best_params)
    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))

[I 2023-07-24 20:36:38,261] A new study created in memory with name: no-name-aa03d252-9d73-4ee6-ab50-84a6b394a82f


"\n    params, train_set, num_boost_round=1000, folds=None, nfold=5, stratified=True, shuffle=True, fobj=None, feval=None, \n    feature_name='auto', categorical_feature='auto', early_stopping_rounds=None, fpreproc=None, verbose_eval=None, show_stdv=True, \n    seed=0, callbacks=None, time_budget=None, sample_size=None, study=None, optuna_callbacks=None, \n    verbosity=None, show_progress_bar=True, model_dir=None, return_cvbooster=False, *, optuna_seed=None)\n    "

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=60, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=60
Training until validation scores don't improve for 100 rounds
[100]	cv_agg's binary_logloss: 0.194632 + 0.0428421	cv_agg's balanced_log_loss: 0.23877 + 0.0585708


feature_fraction, val_score: 0.188233:  14%|#4        | 1/7 [00:00<00:05,  1.05it/s]

...

Early stopping, best iteration is:
[65]	cv_agg's binary_logloss: 0.189601 + 0.034191	cv_agg's balanced_log_loss: 0.218824 + 0.0398688
[100]	cv_agg's binary_logloss: 0.188496 + 0.0523701	cv_agg's balanced_log_loss: 0.228188 + 0.0668956


min_data_in_leaf, val_score: 0.184292:  80%|########  | 4/5 [00:03<00:00,  1.31it/s]

Early stopping, best iteration is:
[65]	cv_agg's binary_logloss: 0.189601 + 0.034191	cv_agg's balanced_log_loss: 0.218824 + 0.0398688
[100]	cv_agg's binary_logloss: 0.188496 + 0.0523701	cv_agg's balanced_log_loss: 0.228188 + 0.0668956


min_data_in_leaf, val_score: 0.184292: 100%|##########| 5/5 [00:03<00:00,  1.30it/s]

Early stopping, best iteration is:
[65]	cv_agg's binary_logloss: 0.189601 + 0.034191	cv_agg's balanced_log_loss: 0.218824 + 0.0398688
Best score: 0.18429226749685973
Best params: {'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 4.1834969007502644e-07, 'lambda_l2': 0.03705451993798532, 'num_leaves': 88, 'feature_fraction': 1.0, 'bagging_fraction': 0.8347595746177183, 'bagging_freq': 5, 'seed': 42, 'use_missing': True, 'feature_pre_filter': False, 'min_data_in_leaf': 60, 'min_child_samples': 20}
  Params: 
    objective: binary
    metric: binary_logloss
    verbosity: -1
    boosting_type: gbdt
    lambda_l1: 4.1834969007502644e-07
    lambda_l2: 0.03705451993798532
    num_leaves: 88
    feature_fraction: 1.0
    bagging_fraction: 0.8347595746177183
    bagging_freq: 5
    seed: 42
    use_missing: True
    feature_pre_filter: False
    min_data_in_leaf: 60
    min_child_samples: 20


# A pruner for LightGBM

In [27]:
import numpy as np
import optuna

import lightgbm as lgb

from sklearn.model_selection import train_test_split


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):

    train_x,valid_x,train_y,valid_y=train_test_split(X_df,y_df,test_size=0.2,random_state=42,shuffle=True,stratify=y_df)

    dtrain = lgb.Dataset(train_x,label=train_y,
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])
    dvalid = lgb.Dataset(valid_x,label=valid_y,
                         feature_name=['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
                                       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
                                       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
                                       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
                                       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL'],
                         categorical_feature=["EJ"])

    param = {"objective": "binary",
             "metric": "binary_logloss",
             "verbosity": -1,
             "boosting_type": "gbdt",
             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
             "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
             "seed":42,
             "use_missing":True}
             
    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "") # https://lightgbm.readthedocs.io/en/latest/Parameters.html#metric

    gbm = lgb.train(param, dtrain, valid_sets=[dvalid],feval=balanced_log_loss_eval,categorical_feature=["EJ"])

    preds = gbm.predict(valid_x)
    
    bll = balanced_log_loss(valid_y,preds)
    
    return bll


if __name__ == "__main__":
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-07-24 20:37:35,292] A new study created in memory with name: no-name-bd975b3f-148d-45f5-a99a-07614203dd0f
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.

[I 2023-07-24 20:37:35,388] Trial 0 finished with value: 0.26150487352511514 and parameters: {'lambda_l1': 0.956453451369732, 'lambda_l2': 2.6500907378687464e-07, 'num_leaves': 160, 'feature_fraction': 0.5093140617292277, 'bagging_fraction': 0.6820991516882042, 'bagging_freq': 3, 'min_child_samples': 37}. Best is trial 0 with value: 0.26150487352511514.


[1]	valid_0's binary_logloss: 0.440406	valid_0's balanced_log_loss: 0.527097
[2]	valid_0's binary_logloss: 0.428045	valid_0's balanced_log_loss: 0.508639
[3]	valid_0's binary_logloss: 0.41824	valid_0's balanced_log_loss: 0.493796
[4]	valid_0's binary_logloss: 0.407979	valid_0's balanced_log_loss: 0.477909
[5]	valid_0's binary_logloss: 0.390981	valid_0's balanced_log_loss: 0.451946
[6]	valid_0's binary_logloss: 0.378506	valid_0's balanced_log_loss: 0.434888
[7]	valid_0's binary_logloss: 0.369964	valid_0's balanced_log_loss: 0.425605
[8]	valid_0's binary_logloss: 0.359649	valid_0's balanced_log_loss: 0.411895
[9]	valid_0's binary_logloss: 0.35126	valid_0's balanced_log_loss: 0.401173
[10]	valid_0's binary_logloss: 0.345091	valid_0's balanced_log_loss: 0.395727
[11]	valid_0's binary_logloss: 0.342181	valid_0's balanced_log_loss: 0.39312
[12]	valid_0's binary_logloss: 0.335751	valid_0's balanced_log_loss: 0.385431
[13]	valid_0's binary_logloss: 0.32844	valid_0's balanced_log_loss: 0.376611

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.

[I 2023-07-24 20:37:35,474] Trial 1 finished with value: 0.29928036131802466 and parameters: {'lambda_l1': 9.838007951666135e-07, 'lambda_l2': 3.7195397745704947e-07, 'num_leaves': 200, 'feature_fraction': 0.6762774251316028, 'bagging_fraction': 0.664975252410102, 'bagging_freq': 3, 'min_child_samples': 91}. Best is trial 1 with value: 0.29928036131802466.
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.



[93]	valid_0's binary_logloss: 0.25559	valid_0's balanced_log_loss: 0.302344
[94]	valid_0's binary_logloss: 0.253635	valid_0's balanced_log_loss: 0.300441
[95]	valid_0's binary_logloss: 0.253032	valid_0's balanced_log_loss: 0.299062
[96]	valid_0's binary_logloss: 0.252168	valid_0's balanced_log_loss: 0.297119
[97]	valid_0's binary_logloss: 0.251747	valid_0's balanced_log_loss: 0.297445
[98]	valid_0's binary_logloss: 0.251629	valid_0's balanced_log_loss: 0.298079
[99]	valid_0's binary_logloss: 0.251324	valid_0's balanced_log_loss: 0.299542
[100]	valid_0's binary_logloss: 0.250472	valid_0's balanced_log_loss: 0.29928
[1]	valid_0's binary_logloss: 0.432726	valid_0's balanced_log_loss: 0.512349
[2]	valid_0's binary_logloss: 0.413574	valid_0's balanced_log_loss: 0.487062
[3]	valid_0's binary_logloss: 0.394974	valid_0's balanced_log_loss: 0.461061
[4]	valid_0's binary_logloss: 0.382992	valid_0's balanced_log_loss: 0.444705
[5]	valid_0's binary_logloss: 0.372706	valid_0's balanced_log_loss: 0

[I 2023-07-24 20:37:35,611] Trial 2 finished with value: 0.25184848534581117 and parameters: {'lambda_l1': 3.6433059104581143e-06, 'lambda_l2': 5.02255151222259e-07, 'num_leaves': 189, 'feature_fraction': 0.8422874360825792, 'bagging_fraction': 0.998711084418835, 'bagging_freq': 5, 'min_child_samples': 50}. Best is trial 1 with value: 0.29928036131802466.


...

[1]	valid_0's binary_logloss: 0.463058	valid_0's balanced_log_loss: 0.557668
[2]	valid_0's binary_logloss: 0.459499	valid_0's balanced_log_loss: 0.551145
[3]	valid_0's binary_logloss: 0.455708	valid_0's balanced_log_loss: 0.541737
[4]	valid_0's binary_logloss: 0.452689	valid_0's balanced_log_loss: 0.533878
[5]	valid_0's binary_logloss: 0.44576	valid_0's balanced_log_loss: 0.521545
[6]	valid_0's binary_logloss: 0.442565	valid_0's balanced_log_loss: 0.51294
[7]	valid_0's binary_logloss: 0.441745	valid_0's balanced_log_loss: 0.511463
[8]	valid_0's binary_logloss: 0.441208	valid_0's balanced_log_loss: 0.510323
[9]	valid_0's binary_logloss: 0.440259	valid_0's balanced_log_loss: 0.513431
[10]	valid_0's binary_logloss: 0.436612	valid_0's balanced_log_loss: 0.512431
[11]	valid_0's binary_logloss: 0.436612	valid_0's balanced_log_loss: 0.512431
[12]	valid_0's binary_logloss: 0.433432	valid_0's balanced_log_loss: 0.509953
[13]	valid_0's binary_logloss: 0.430508	valid_0's balanced_log_loss: 0.5074

[I 2023-07-24 20:37:47,858] Trial 96 finished with value: 0.3012334881612167 and parameters: {'lambda_l1': 1.4965430573141072, 'lambda_l2': 0.08694753960299846, 'num_leaves': 138, 'feature_fraction': 0.9673026422153682, 'bagging_fraction': 0.45931183482697974, 'bagging_freq': 2, 'min_child_samples': 57}. Best is trial 67 with value: 0.5054416204102644.
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.

[I 2023-07-24 20:37:47,992] Trial 97 finished with value: 0.4087146305035067 and parameters: {'lambda_l1': 5.417551576804896, 'lambda_l2': 0.03601810580052642, 'num_leaves': 117, 'feature_fraction': 0.9536467646632587, 'bagging_fraction': 0.4466000259089464, 'bagging_freq': 1, 'min_child_samples': 98}. Best is trial 67 with value: 0.5054416204102644.
/opt/conda/lib/python3.10/site-packa

[1]	valid_0's binary_logloss: 0.463451	valid_0's balanced_log_loss: 0.557831
[2]	valid_0's binary_logloss: 0.464025	valid_0's balanced_log_loss: 0.555434
[3]	valid_0's binary_logloss: 0.459242	valid_0's balanced_log_loss: 0.543856
[4]	valid_0's binary_logloss: 0.45797	valid_0's balanced_log_loss: 0.540283
[5]	valid_0's binary_logloss: 0.454374	valid_0's balanced_log_loss: 0.539116
[6]	valid_0's binary_logloss: 0.452793	valid_0's balanced_log_loss: 0.540286
[7]	valid_0's binary_logloss: 0.449253	valid_0's balanced_log_loss: 0.536793
[8]	valid_0's binary_logloss: 0.446979	valid_0's balanced_log_loss: 0.53348
[9]	valid_0's binary_logloss: 0.443287	valid_0's balanced_log_loss: 0.522549
[10]	valid_0's binary_logloss: 0.437594	valid_0's balanced_log_loss: 0.514211
[11]	valid_0's binary_logloss: 0.436564	valid_0's balanced_log_loss: 0.515149
[12]	valid_0's binary_logloss: 0.434633	valid_0's balanced_log_loss: 0.516526
[13]	valid_0's binary_logloss: 0.432492	valid_0's balanced_log_loss: 0.5137

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.

[I 2023-07-24 20:37:48,256] Trial 99 finished with value: 0.3258660824600317 and parameters: {'lambda_l1': 2.346101927730311, 'lambda_l2': 0.004270200817761386, 'num_leaves': 61, 'feature_fraction': 0.9379702600377371, 'bagging_fraction': 0.4795343466029177, 'bagging_freq': 1, 'min_child_samples': 88}. Best is trial 67 with value: 0.5054416204102644.


[1]	valid_0's binary_logloss: 0.46075	valid_0's balanced_log_loss: 0.555982
[2]	valid_0's binary_logloss: 0.460038	valid_0's balanced_log_loss: 0.551468
[3]	valid_0's binary_logloss: 0.451847	valid_0's balanced_log_loss: 0.533741
[4]	valid_0's binary_logloss: 0.449152	valid_0's balanced_log_loss: 0.5301
[5]	valid_0's binary_logloss: 0.445171	valid_0's balanced_log_loss: 0.524798
[6]	valid_0's binary_logloss: 0.440763	valid_0's balanced_log_loss: 0.521894
[7]	valid_0's binary_logloss: 0.436279	valid_0's balanced_log_loss: 0.518575
[8]	valid_0's binary_logloss: 0.433077	valid_0's balanced_log_loss: 0.514798
[9]	valid_0's binary_logloss: 0.429421	valid_0's balanced_log_loss: 0.503175
[10]	valid_0's binary_logloss: 0.425773	valid_0's balanced_log_loss: 0.497614
[11]	valid_0's binary_logloss: 0.424064	valid_0's balanced_log_loss: 0.498056
[12]	valid_0's binary_logloss: 0.421555	valid_0's balanced_log_loss: 0.499308
[13]	valid_0's binary_logloss: 0.413571	valid_0's balanced_log_loss: 0.48952

# Submission

In [28]:
'''
# Load or prepare the unseen data
col = ["EJ"] 
submission_df = test_df.drop(["Id"], axis=1)

# Apply label encoder to each column
submission_df[col] = submission_df[col].apply(le.fit_transform) 
ids = test_df['Id'] # list or array of shape (5,) containing the IDs

# Get the best hyperparameters from the Optuna study
best_params = study.best_params

# Create a LightGBM dataset from the entire training data
dtrain = lgb.Dataset(X_df, label=y_df)

# Train a LightGBM model using the best hyperparameters
gbm = lgb.train(best_params, dtrain)

# Make predictions on unseen data
preds = gbm.predict(submission_df)

# The predictions are probabilities for class 1
prob_class_1 = preds

# To get probabilities for class 0, subtract from 1
prob_class_0 = 1 - prob_class_1

# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946
# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409801
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
sample['class_1'] = prob_class_1
sample['class_0'] = prob_class_0
sample.to_csv('submission.csv', index = False)
'''

'\n# Load or prepare the unseen data\ncol = ["EJ"] \nsubmission_df = test_df.drop(["Id"], axis=1)\n\n# Apply label encoder to each column\nsubmission_df[col] = submission_df[col].apply(le.fit_transform) \nids = test_df[\'Id\'] # list or array of shape (5,) containing the IDs\n\n# Get the best hyperparameters from the Optuna study\nbest_params = study.best_params\n\n# Create a LightGBM dataset from the entire training data\ndtrain = lgb.Dataset(X_df, label=y_df)\n\n# Train a LightGBM model using the best hyperparameters\ngbm = lgb.train(best_params, dtrain)\n\n# Make predictions on unseen data\npreds = gbm.predict(submission_df)\n\n# The predictions are probabilities for class 1\nprob_class_1 = preds\n\n# To get probabilities for class 0, subtract from 1\nprob_class_0 = 1 - prob_class_1\n\n# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946\n# https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409801\nsample = pd.re

# References
- https://optuna.readthedocs.io/en/stable/index.html
- https://github.com/optuna/optuna-examples/blob/main/lightgbm/lightgbm_simple.py
- https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/412946
- https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409801